In [1]:
from pathlib import Path
import pandas as pd

In [2]:
home_dir = Path.home()
inside_airbnb_data_dir = home_dir / 'Programming/data/inside-airbnb/london'
population_dataset_england_wales_data_dir = home_dir / 'Programming/data/office-national-statistics/population-dataset-england-wales/'
crime_rate_dir = home_dir / 'Programming/data/crime-rate/'

In [3]:
inside_airbnb_data_file = inside_airbnb_data_dir / 'listings.csv'
population_dataset_england_wales_data_file = population_dataset_england_wales_data_dir / 'mye23tablesew.xlsx'
crime_rate_data_file = crime_rate_dir / 'crimerate-pro-data-table-rmp-region-towns-cities.csv'

In [4]:
crime_rate_df = pd.read_csv(crime_rate_data_file, usecols=['Borough', 'Crime Rate'])
crime_rate_df.rename(columns={'Borough': 'borough', 'Crime Rate': 'crime_rate'}, inplace=True)
crime_rate_df.set_index('borough', inplace=True)
crime_rate_df.drop('DownloadCSVExcelTSV', inplace=True)

In [5]:
columns_list = ['neighbourhood_cleansed', 'bathrooms', 'bedrooms', 'latitude', 'longitude',
                'room_type', 'latitude', 'longitude', 'property_type', 'price']
inside_airbnb_df = pd.read_csv(inside_airbnb_data_file, usecols=columns_list)
inside_airbnb_df.rename(columns={'neighbourhood_cleansed': 'borough'}, inplace=True)
inside_airbnb_df.price = inside_airbnb_df.price.str.replace('$', '')
inside_airbnb_df.head()

,borough,latitude,longitude,property_type,room_type,bathrooms,bedrooms,price
0,Hammersmith and Fulham,51.49392,-0.22754,Entire rental unit,Entire home/apt,1.5,1.0,89.00
1,Tower Hamlets,51.52435,-0.06938,Entire condo,Entire home/apt,3.0,3.0,220.00
2,Hammersmith and Fulham,51.49547,-0.22864,Entire rental unit,Entire home/apt,1.0,1.0,88.00
3,Hammersmith and Fulham,51.49368,-0.22774,Room in aparthotel,Private room,1.0,1.0,75.00
4,Haringey,51.59031,-0.09408,Private room in rental unit,Private room,1.0,1.0,53.00


In [6]:
inside_airbnb_df.shape

(95144, 8)

In [7]:
crime_rate_df.shape

(33, 1)